In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import arviz as az
import matplotlib.pyplot as plt
import scipy.linalg as la
import scipy.stats as st
from tqdm import trange

from functions.load_data import load_data
from functions.gibbs_reg import gibbs_regression
from functions.mcmc_stats import mcmc_stats
from functions.mcmc_plots import mcmc_plots
from functions.plot_posterior_vs_prior import prior_vs_posterior_plot
from functions.sddr_beta import sddr_beta

data = load_data()

/Users/satoshan/.pyenv/versions/miniforge3-4.10.3-10/envs/numpyro-env/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


$$\pi_t = \alpha \pi_{t-1} + \beta E_{t}\pi_{t+1} + \kappa Y_{t} + \gamma \Delta\text{oil}_{t} + \varepsilon_t$$

$E_{t}\pi_{t+1}$:

head line inflation expectation (model’s output)

$Y_t$: Output gap


$\Delta\text{oil}$: 

Year on year change in log oil prices (WTI oil price index), Aggregated method: Avarage

In [2]:
# ==== priors ====
tau  = 1.0
nu0  = 12.0
lam0 = 12.0

# ==== data ====
targets   = ["pi_cpi", "pi_cpi_core", "pi_pce", "pi_pce_core"]  
epi_specs = ["Epi", "Epi_spf_gdp", "Epi_spf_cpi"]  
def _lag_var_for(target):
    return f"{target}_prev"

def _prepare_X_y(df, target, epi_col, output_var):
    lag_col = _lag_var_for(target)
    cols = [lag_col, epi_col, output_var, "oil"]
    tmp = df[[target] + cols].dropna().copy()
    y = tmp[target].to_numpy()
    X = tmp[cols].copy()
    return X, y, tmp.index, cols

def _ols_fit(X, y):
    model = sm.OLS(y, X)  
    res = model.fit()
    out = (pd.DataFrame({
        "coef": res.params,
        "std_err": res.bse,
        "t": res.tvalues,
        "p": res.pvalues
    })
           .reset_index()
           .rename(columns={"index": "param"}))
    return out, res

def _gibbs_fit(X, y, draws=20000, burnin=2000, seed=1234):
    k = X.shape[1]
    A0 = np.eye(k) / (tau**2)
    b0 = np.zeros(k)
    hyper_param = [b0, A0, nu0, lam0]
    runs = gibbs_regression(y, X.to_numpy(), hyper_param, burnin, draws, rng=np.random.default_rng(seed))
    beta = runs[burnin:, :k]
    mean = beta.mean(0)
    lo, hi = np.quantile(beta, [0.025, 0.975], axis=0)
    out = pd.DataFrame({
        "param": X.columns,
        "post_mean": mean,
        "ci_lower": lo,
        "ci_upper": hi
    })
    return out, runs


In [3]:
ols_rows   = []
gibbs_rows = []

for target in targets:
    for epi in epi_specs:
        X, y, used_idx, cols = _prepare_X_y(data, target, "output_gap_BN", epi)
        # OLS
        ols_tbl, _ = _ols_fit(X, y)
        ols_tbl.insert(0, "target", target)
        ols_tbl.insert(1, "Epi_spec", epi)
        ols_rows.append(ols_tbl)
        # Gibbs
        gibbs_tbl, runs = _gibbs_fit(X, y, draws=20000, burnin=2000, seed=1234)
        gibbs_tbl.insert(0, "target", target)
        gibbs_tbl.insert(1, "Epi_spec", epi)
        gibbs_rows.append(gibbs_tbl)

ols_results   = pd.concat(ols_rows, ignore_index=True)
gibbs_results = pd.concat(gibbs_rows, ignore_index=True)
ols_results   = ols_results[["target","Epi_spec","param","coef","std_err","t","p"]]
gibbs_results = gibbs_results[["target","Epi_spec","param","post_mean","ci_lower","ci_upper"]]

print("=== OLS: output_gap_BN ===")
ols_gap_coef = (
    ols_results.query("param == 'output_gap_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="coef")
    .round(3)
)
ols_gap_p = (
    ols_results.query("param == 'output_gap_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="p")
    .round(3)
)
ols_gap_coef.index = ["coef"]
ols_gap_p.index    = ["p"]
ols_gap_two_rows = pd.concat([ols_gap_coef, ols_gap_p])
display(ols_gap_two_rows)

print("\n=== Gibbs results (posterior mean) ===")
gibbs_gap = (
    gibbs_results.query("param == 'output_gap_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="post_mean")
    .round(3)
)
display(gibbs_gap)

100%|██████████| 22000/22000 [00:00<00:00, 23760.71it/s]

=== OLS: output_gap_BN ===


target   pi_cpi                         pi_cpi_core                          \
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
coef     -0.012       0.016       0.011       0.023       0.038       0.029   
p         0.675       0.593       0.706       0.263       0.048       0.112   

target   pi_pce                         pi_pce_core                          
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
coef     -0.006       0.003       0.009       0.003       0.006       0.010  
p         0.755       0.875       0.640       0.795       0.654       0.428


=== Gibbs results (posterior mean) ===


target        pi_cpi                         pi_cpi_core              \
Epi_spec         Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp   
param                                                                  
output_gap_BN -0.011       0.018       0.012       0.022       0.038   

target                    pi_pce                         pi_pce_core  \
Epi_spec      Epi_spf_cpi    Epi Epi_spf_gdp Epi_spf_cpi         Epi   
param                                                                  
output_gap_BN       0.028 -0.006       0.004        0.01       0.003   

target                                 
Epi_spec      Epi_spf_gdp Epi_spf_cpi  
param                                  
output_gap_BN       0.006       0.009

In [4]:
ols_rows   = []
gibbs_rows = []

for target in targets:
    for epi in epi_specs:
        X, y, used_idx, cols = _prepare_X_y(data, target, "unemp_gap", epi)
        # OLS
        ols_tbl, _ = _ols_fit(X, y)
        ols_tbl.insert(0, "target", target)
        ols_tbl.insert(1, "Epi_spec", epi)
        ols_rows.append(ols_tbl)
        # Gibbs
        gibbs_tbl, runs = _gibbs_fit(X, y, draws=20000, burnin=2000, seed=1234)
        gibbs_tbl.insert(0, "target", target)
        gibbs_tbl.insert(1, "Epi_spec", epi)
        gibbs_rows.append(gibbs_tbl)

ols_results   = pd.concat(ols_rows, ignore_index=True)
gibbs_results = pd.concat(gibbs_rows, ignore_index=True)
ols_results   = ols_results[["target","Epi_spec","param","coef","std_err","t","p"]]
gibbs_results = gibbs_results[["target","Epi_spec","param","post_mean","ci_lower","ci_upper"]]

print("=== OLS: unemp_gap ===")
ols_gap_coef = (
    ols_results.query("param == 'unemp_gap'")
    .pivot(index="param", columns=["target","Epi_spec"], values="coef")
    .round(3)
)
ols_gap_p = (
    ols_results.query("param == 'unemp_gap'")
    .pivot(index="param", columns=["target","Epi_spec"], values="p")
    .round(3)
)
ols_gap_coef.index = ["coef"]
ols_gap_p.index    = ["p"]
ols_gap_two_rows = pd.concat([ols_gap_coef, ols_gap_p])
display(ols_gap_two_rows)

print("\n=== Gibbs results (posterior mean) ===")
gibbs_gap = (
    gibbs_results.query("param == 'unemp_gap'")
    .pivot(index="param", columns=["target","Epi_spec"], values="post_mean")
    .round(3)
)
display(gibbs_gap)

100%|██████████| 22000/22000 [00:01<00:00, 16717.86it/s]

=== OLS: unemp_gap ===


target   pi_cpi                         pi_cpi_core                          \
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
coef      0.002       0.049       0.034       0.024       0.051       0.040   
p         0.923       0.053       0.156       0.109       0.001       0.005   

target   pi_pce                         pi_pce_core                          
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
coef     -0.035      -0.002      -0.016      -0.009       0.002      -0.002  
p         0.044       0.922       0.358       0.440       0.833       0.870


=== Gibbs results (posterior mean) ===


target    pi_cpi                         pi_cpi_core                          \
Epi_spec     Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
param                                                                          
unemp_gap  0.004        0.05       0.035       0.024       0.051        0.04   

target    pi_pce                         pi_pce_core                          
Epi_spec     Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
param                                                                         
unemp_gap -0.035      -0.001      -0.015      -0.009       0.002      -0.002

In [5]:
ols_rows   = []
gibbs_rows = []

for target in targets:
    for epi in epi_specs:
        X, y, used_idx, cols = _prepare_X_y(data, target, "markup_BN_inv", epi)
        # OLS
        ols_tbl, _ = _ols_fit(X, y)
        ols_tbl.insert(0, "target", target)
        ols_tbl.insert(1, "Epi_spec", epi)
        ols_rows.append(ols_tbl)
        # Gibbs
        gibbs_tbl, runs = _gibbs_fit(X, y, draws=20000, burnin=2000, seed=1234)
        gibbs_tbl.insert(0, "target", target)
        gibbs_tbl.insert(1, "Epi_spec", epi)
        gibbs_rows.append(gibbs_tbl)

ols_results   = pd.concat(ols_rows, ignore_index=True)
gibbs_results = pd.concat(gibbs_rows, ignore_index=True)
ols_results   = ols_results[["target","Epi_spec","param","coef","std_err","t","p"]]
gibbs_results = gibbs_results[["target","Epi_spec","param","post_mean","ci_lower","ci_upper"]]

print("=== OLS: markup_BN_inv ===")
ols_gap_coef = (
    ols_results.query("param == 'markup_BN_inv'")
    .pivot(index="param", columns=["target","Epi_spec"], values="coef")
    .round(3)
)
ols_gap_p = (
    ols_results.query("param == 'markup_BN_inv'")
    .pivot(index="param", columns=["target","Epi_spec"], values="p")
    .round(3)
)
ols_gap_coef.index = ["coef"]
ols_gap_p.index    = ["p"]
ols_gap_two_rows = pd.concat([ols_gap_coef, ols_gap_p])
display(ols_gap_two_rows)

print("\n=== Gibbs results (posterior mean) ===")
gibbs_gap = (
    gibbs_results.query("param == 'markup_BN_inv'")
    .pivot(index="param", columns=["target","Epi_spec"], values="post_mean")
    .round(3)
)
display(gibbs_gap)

100%|██████████| 22000/22000 [00:00<00:00, 22468.81it/s]

=== OLS: markup_BN_inv ===


target   pi_cpi                         pi_cpi_core                          \
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
coef      0.044       0.082       0.080       0.044       0.070       0.073   
p         0.484       0.219       0.204       0.261       0.091       0.059   

target   pi_pce                         pi_pce_core                          
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
coef     -0.028       0.001      -0.011       0.001       0.011       0.007  
p         0.539       0.982       0.814       0.958       0.692       0.800


=== Gibbs results (posterior mean) ===


target        pi_cpi                         pi_cpi_core              \
Epi_spec         Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp   
param                                                                  
markup_BN_inv  0.042       0.079       0.077       0.043       0.069   

target                    pi_pce                         pi_pce_core  \
Epi_spec      Epi_spf_cpi    Epi Epi_spf_gdp Epi_spf_cpi         Epi   
param                                                                  
markup_BN_inv       0.072 -0.029       0.001      -0.011       0.001   

target                                 
Epi_spec      Epi_spf_gdp Epi_spf_cpi  
param                                  
markup_BN_inv       0.012       0.007